## Imports

In [2]:
from scipy.cluster import hierarchy as hc
from scipy.stats.mstats import winsorize
from scipy.interpolate import interp1d
from sklearn import linear_model
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from random import sample
import networkx as nx
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
import functions
import scipy.io

matplotlib.rcParams['figure.figsize'] = (20, 20);

## Data

In [2]:
mat, data = functions.load_and_filter_data('/Users/kitovh/Documents/Research/PlasmaResearch/\
Cristina_Group/DIII_D_data/d3d-db-220420.mat', Filter=True)

data = data[data.isna().apply(all)[data.isna().apply(all) == False].index]
data['intentional_disruption'] = data['intentional_disruption'].fillna(-1)

In [3]:
"""
Filter for longer lasting shots
"""
counts = data.groupby('shot')['time'].count()
print(counts.mean())
shots = counts[counts >= counts.mean()]
data = data[data.shot.isin(shots.index)]

227.86681766704416


In [4]:
"""
Filter for shots with positive IP 
"""
bad_shots = []
shots = []
for shot in tqdm(data.shot.unique()):
    mean = data[data.shot == shot]['ip'].mean()
    mini = data[data.shot == shot]['ip'].values[-1]
    
    if mini > mean:
        bad_shots.append(shot)
    else:
        shots.append(shot)

data = data[data.shot.isin(shots)]

100%|██████████| 7991/7991 [00:33<00:00, 235.73it/s]


In [5]:
"""
Filter features based on shot coverage
"""
coverage = 100 * data.groupby('shot').apply(lambda x: x.count() / x.shape[0])
coverage = coverage.mean()
coverage = coverage[coverage >= 90]

if 'time_until_disrupt' not in coverage.index:
    columns = ['time_until_disrupt'] + list(coverage.index)
    
data = data[columns]

In [6]:
"""
Filter shots based on feature coverage
"""
shots = []
for shot in tqdm(data.shot.unique()):
    shot_data = data[data.shot == shot].copy()
    shot_data_columns = [a for a in shot_data.columns if 
                          (a != 'intentional_disruption')
                        & (a != 'time_until_disrupt')]
    
    shot_data = shot_data[shot_data_columns]
    shot_coverage = 100 * (shot_data.count() / shot_data.shape[0])

    if np.mean(shot_coverage) > 90:
        shots.append(shot)
data = data[data.shot.isin(shots)]

100%|██████████| 7623/7623 [00:36<00:00, 206.86it/s]


In [7]:
"""
Remove Ramp-up Phase of shots
"""
filtered_data = []
for shot in tqdm(data.shot.unique()):
    df = data[data.shot == shot]
    df = df.reset_index(drop=True)

    mode_value = df['dipprog_dt'].mode().values[0]

    if df[df['dipprog_dt'] == mode_value].index[0] / df.index.max() <= 0.75:
        df = df[df[df['dipprog_dt'] == mode_value].index[0]:]
        filtered_data.append(df)

data = pd.concat(filtered_data)
data.reset_index(inplace=True, drop=True)

100%|██████████| 7112/7112 [00:24<00:00, 292.27it/s]


In [8]:
"""
Filter shots that have high variation in Delta IP 
- This is because some shots just look like inverted triangles (no/little flattop) and are pointless to keep
"""

bad_shots = []
shots = []
for i in tqdm(range(len(data.shot.unique()))):
    df = data[data.shot == data.shot.unique()[i]]
    df.reset_index(drop=True, inplace=True)
    
    if np.abs(df['dipprog_dt']).min() < 1000:
        shots.append(data.shot.unique()[i])
    else:
        bad_shots.append(data.shot.unique()[i])
data = data[data.shot.isin(shots)]

100%|██████████| 6982/6982 [04:12<00:00, 27.63it/s]


In [9]:
"""
Remove shots with programmed ramp down 
"""
activate = False
if activate == True:
    bad_shots = []
    shots = []
    for shot in tqdm(data.shot.unique()):
        df = data[data.shot == shot]
        counts = (df['dipprog_dt'] < 0).value_counts()
        counts = counts / counts.sum()

        if True in counts.index:
            bad_shots.append(shot)
        else:
            shots.append(shot)

In [10]:
comparison = data[['shot', 'time_until_disrupt']]
comparison = comparison.groupby('shot').min()
comparison['variable'] = ['disrupt' if a == 0 else 'no_disrupt' for a in comparison['time_until_disrupt']]
comparison = comparison['variable'].reset_index()
comparison = comparison[comparison.shot.isin(data.shot.unique())]

In [11]:
# cols = [a for a in data if (a != 'time') & (a != 'shot') & (a != 'time_until_disrupt')]
# data[cols] = (data[cols] - data[cols].mean()) / data[cols].std()

In [12]:
# matplotlib.rcParams['figure.figsize'] = (20, 50);

# shot_number = comparison[comparison.variable == 'disrupt'].shot.unique()[0]
# fig, axs = plt.subplots(len(data.columns))
# for i in range(len(data.columns)): 
#     axs[i].plot(data[data.shot == shot_number][data.columns[i]])
#     axs[i].set_title(data.columns[i])
# plt.show()

In [13]:
# fig = plt.figure(figsize = (15, 20))
# ax = fig.gca()
# data.hist(ax=ax);

In [108]:
training_non_disrupt = sample(list(comparison[comparison['variable'] == 'no_disrupt']['shot'].values), 300)
training_disrupt = sample(list(comparison[comparison['variable'] == 'disrupt']['shot'].values), 350)

training_data = data[data.shot.isin(training_non_disrupt) | data.shot.isin(training_disrupt)]

In [111]:
testing_values = comparison[~comparison.shot.isin(training_data.shot.unique())]

testing_non_disrupt = sample(list(testing_values[testing_values['variable'] == 'no_disrupt']['shot'].values), 10)
testing_disrupt = sample(list(testing_values[testing_values['variable'] == 'disrupt']['shot'].values), 10)

testing_data = data[data.shot.isin(testing_non_disrupt) | data.shot.isin(testing_disrupt)]

In [112]:
# plt.plot(training_data[training_data.shot == training_data.shot.unique()[0]]['ip'].values)
# plt.plot(training_data[training_data.shot == training_data.shot.unique()[1]]['ip'].values)
# plt.plot(training_data[training_data.shot == training_data.shot.unique()[2]]['ip'].values)
# plt.plot(training_data[training_data.shot == training_data.shot.unique()[3]]['ip'].values)
# plt.plot(training_data[training_data.shot == training_data.shot.unique()[4]]['ip'].values)
# plt.plot(training_data[training_data.shot == training_data.shot.unique()[5]]['ip'].values)
# plt.plot(training_data[training_data.shot == training_data.shot.unique()[6]]['ip'].values)

In [113]:
shot_numbers = training_non_disrupt + training_disrupt + testing_non_disrupt + testing_disrupt


In [115]:
shot_numbers = training_non_disrupt + training_disrupt + testing_non_disrupt + testing_disrupt

training_interpolated_data = []
testing_interpolated_data = []

for shot in tqdm(shot_numbers):
    shot_data_backup = data[data['shot'] == shot]

    granularity = 0.0001
    x = np.arange(shot_data_backup['time'].min(), shot_data_backup['time'].max(), step=granularity)
    x = x[x <= shot_data_backup['time'].max()]

    features = [a for a in data.columns if 
                 (a != 'shot') & (a != 'time')
               & (a != 'time_until_disrupt') & (a != 'other_hardware_failure')
               & (a != 'power_supply_railed') & (a != 'intentional_disruption')]

    interpolated = pd.DataFrame({'time':x})
    for feature in features:
        f1 = interp1d(shot_data_backup['time'], shot_data_backup[feature])
        interpolated[feature] = f1(x)

    interpolated = interpolated.rename(columns={'ip':'interpolated_ip'})
    interpolated['shot'] = shot
    
    if shot in training_data.shot.unique():
        training_interpolated_data.append(interpolated)
        
    elif shot in testing_data.shot.unique():
        testing_interpolated_data.append(interpolated)

100%|██████████| 670/670 [00:27<00:00, 24.71it/s]


In [116]:
offset = 0.04 / 0.0001
print(offset)

400.0


In [118]:
training_data = pd.concat(training_interpolated_data)
testing_data = pd.concat(testing_interpolated_data)

In [119]:
coverage_percent = (training_data.isna().sum() / training_data.shape[0]) * 100
coverage_percent = coverage_percent[coverage_percent <= 5]

In [121]:
training_data = training_data[coverage_percent.index]
testing_data = testing_data[coverage_percent.index]

In [125]:
training_data.set_index('shot', inplace=True)
testing_data.set_index('shot', inplace=True)

In [127]:
returned_training_data = []
for shot in training_data.index.unique():
    returned_training_data.append(
        training_data[training_data.index == shot].fillna(method='ffill')
    )
    
returned_testing_data = []
for shot in testing_data.index.unique():
    returned_testing_data.append(
        testing_data[testing_data.index == shot].fillna(method='ffill')
    )

In [132]:
training_data = pd.concat(returned_training_data)
testing_data = pd.concat(returned_testing_data)

In [133]:
training_data.reset_index(inplace=True)
testing_data.reset_index(inplace=True)

In [164]:
training_data.dropna(inplace=True)
testing_data.dropna(inplace=True)

In [165]:
cols = [a for a in training_data if (a != 'time') & (a != 'shot')]
cols = [
    'Greenwald_fraction_RT',
    'Te_HWHM',
    'Wmhd_RT',
    'dipprog_dt_RT',
    'interpolated_ip',
    'ip_error_RT',
    'ip_prog',
    'kappa_area',
    'n_e_RT',
    'n_equal_1_normalized',
    'p_rad',
    'radiated_fraction',
    'v_loop',
    'zcur']
cols = list(set(cols).intersection(training_data.columns))
cols.sort()

In [167]:
testing_data[cols] = (testing_data[cols] - training_data[cols].mean()) / training_data[cols].std()
training_data[cols] = (training_data[cols] - training_data[cols].mean()) / training_data[cols].std()

In [168]:
training_data = training_data[['shot', 'time'] + cols]
testing_data = testing_data[['shot', 'time'] + cols]

In [169]:
training_data['time'] = training_data.groupby('shot')['time'].apply(lambda x: x - x.values[0])
testing_data['time'] = testing_data.groupby('shot')['time'].apply(lambda x: x - x.values[0])

<ipython-input-169-3ea2ffd52685>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['time'] = training_data.groupby('shot')['time'].apply(lambda x: x - x.values[0])


In [170]:
# matplotlib.rcParams['figure.figsize'] = (20, 50);

# shot_number = training_disrupt[0]
# fig, axs = plt.subplots(len(training_data.columns))
# for i in range(len(training_data.columns)): 
#     axs[i].plot(training_data[training_data.shot == shot_number][training_data.columns[i]])
#     axs[i].set_title(training_data.columns[i])
# plt.show()

In [171]:
# training_data = training_data.set_index('shot')
# training_data = training_data.groupby('shot').shift(-40)
# training_data.reset_index(inplace=True)

In [172]:
training_data = training_data.set_index('shot')
testing_data = testing_data.set_index('shot')

In [173]:
# training_data.reset_index(inplace=True)
# testing_data.reset_index(inplace=True)

In [174]:
training_data.index = training_data.index / 100000
testing_data.index = testing_data.index / 100000

In [175]:
input_columns = ['time'] + cols
output_columns = ['interpolated_ip', 'ip_error_RT']

In [176]:
training_input = training_data[input_columns].groupby('shot').shift(offset)
training_input.reset_index(inplace=True)

training_output = training_data[output_columns].groupby('shot').shift(-offset)
training_output.reset_index(inplace=True)

In [ ]:
training_input.dropna(inplace=True)
training_output.dropna(inplace=True)

In [ ]:
testing_input = testing_data[input_columns].groupby('shot').shift(offset)
testing_input.reset_index(inplace=True)

testing_output = testing_data[output_columns].groupby('shot').shift(-offset)
testing_output.reset_index(inplace=True)

In [ ]:
testing_input.dropna(inplace=True)
testing_output.dropna(inplace=True)

In [41]:
training_input = training_input[['shot'] + cols].values
training_output = training_output.values

testing_input = testing_input[['shot'] + cols].values
testing_output = testing_output.values

In [42]:
tmp = training_data.copy()
tmp['index'] = np.arange(0, tmp.shape[0])
tmp['shot'] = tmp.index

training_indices = tmp[['shot', 'index']]
training_indices.reset_index(drop=True, inplace=True)
training_indices = training_indices.groupby('shot').index.max().values
training_indices = np.insert(training_indices, 0, 0)

In [43]:
tmp = testing_data.copy()
tmp['index'] = np.arange(0, tmp.shape[0])
tmp['shot'] = tmp.index

testing_indices = tmp[['shot', 'index']]
testing_indices.reset_index(drop=True, inplace=True)
testing_indices = testing_indices.groupby('shot').index.max().values
testing_indices = np.insert(testing_indices, 0, 0)

## Reservoir

In [44]:
def _update_state(A, W_in, state, inputs):
    """
    Computes the next network states by applying the recurrent weights
    to the last state & and feeding in the current input patterns
    Following r(t+1) = tanh{A * r(t) + W_in * u(t)}
        
    Args:
    ----
        state: The preview states.
        input_pattern: Next Intputs
    """
    
    preactivation = (np.dot(A, state) + np.dot(W_in, inputs))
    NextState = np.tanh(preactivation)  
    
    return(NextState)

In [46]:
ReservoirSize = 50
AdjacentMatrixRadius = 0.1
Seed = 1

np.random.seed(1234567)
W = np.random.normal(scale=1, size=(ReservoirSize, ReservoirSize))
OriginalW = W.copy()

# G = nx.fast_gnp_random_graph(ReservoirSize, 
#                              0.01, 
#                              seed=12345)
G = nx.watts_strogatz_graph(n=ReservoirSize,
                            k=5, p=0.1, seed=12345)
W = np.asarray([a * b for a, b in zip(W, nx.adjacency_matrix(G).toarray())])

print("W Standard Deviation:", np.std(W))
radius = np.max(np.abs(np.linalg.eigvals(W)))
A = W.copy()
A = A * AdjacentMatrixRadius / radius
print('A Standard deviation:', np.std(A))

values = []
for i in range(len(G.degree)):
    values.append(G.degree[i])
print("Mean Degree:", np.mean(values))
print("Max Eigenval:", np.max(np.abs(np.linalg.eigvals(A))))

W Standard Deviation: 0.2963465197751988
A Standard deviation: 0.011989522554357283
Mean Degree: 4.0
Max Eigenval: 0.09999999999999981


In [1]:
# nx.draw(G)

In [2]:
# matplotlib.rcParams['figure.figsize'] = (20, 10)
# plt.plot(A, 'o');

In [49]:
# Initialize for states evolution
np.random.seed(12345)
W_in = np.random.normal(scale=0.01, size=(ReservoirSize, training_input.shape[1]-1))
r_matrix = np.zeros((training_input.shape[0], ReservoirSize))

# Evolve states matrix r. 
for t in tqdm(range(1, training_input.shape[0])):
    r_matrix[t, :] = _update_state(
        A=A, 
        W_in=W_in, 
        state=r_matrix[(t)], 
        inputs=training_input[(t-1), 1:]
    )

100%|██████████| 31090940/31090940 [05:28<00:00, 94670.45it/s] 


In [50]:
# plt.plot(r_matrix[:100000, 0])
# plt.plot(r_matrix[100000:200000, 0]);

In [51]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

In [52]:
df = pd.DataFrame(training_input)
df = df.rename(columns={df.columns[0]:'shot'})
df['index'] = np.arange(0, df.shape[0])

indices_end = df.groupby('shot')['index'].last().values
indices_start = df.groupby('shot')['index'].first().values
len(indices_start)

628

In [1015]:
all_models = []
predictions = []
accuracy = [] 
true_values = []

for i in tqdm(range(0, len(indices_start))):
    start = indices_start[i] + 1
    end = indices_end[i] + 1

    model = linear_model.Ridge(alpha=0.1)
    model.fit(
        X=r_matrix[start:end],
        y=training_output[start:end, 1:]
    )
    all_models.append(model)

    prediction = model.predict(
        r_matrix[start:end]
    )
    predictions.append(prediction)

    true_values.append(
        training_output[start:end, 1:]
    )

    accuracy.append(
        mean_squared_error(
            y_true=training_output[start:end, 1:], 
            y_pred=prediction
        )
    )

100%|██████████| 247/247 [00:20<00:00, 11.80it/s]


In [1016]:
# cols = ['time'] + cols

In [4]:
# m = 1
# for i in range(predictions[m].shape[1]):
#     plt.plot(predictions[m][:, i], label='pred')
#     plt.plot(true_values[m][:, i], label='true', alpha=0.5)
#     plt.legend()
#     plt.title(['ip'])
#     plt.show();

In [1018]:
print('mean: ', np.mean(accuracy))
print('median: ', np.median(accuracy))
print('std: ', np.std(accuracy))

mean:  0.011889189298322203
median:  0.007216005169737494
std:  0.017547540878806413


In [3]:
# plt.hist(accuracy, bins=100);

In [1020]:
from sklearn.metrics import mean_squared_error

In [1021]:
df = pd.DataFrame(testing_input)
df = df.rename(columns={df.columns[0]:'shot'})
df['shot'] = df['shot'] * 100000
df['shot'] = df['shot'].astype(int)
df['index'] = np.arange(0, df.shape[0])

indices_end = df.groupby('shot')['index'].last().values
indices_start = df.groupby('shot')['index'].first().values

indices_start.sort()
indices_end.sort()

In [1022]:
# indices_start = np.asarray(indices_start[:1].tolist() + indices_start[-1:].tolist())
# indices_end = np.asarray(indices_end[:1].tolist() + indices_end[-1:].tolist())

In [1023]:
# indices_start = indices_start[-3:]
# indices_end = indices_end[-3:]

In [1024]:
# indices_start = indices_start[-5:]
# indices_end = indices_end[-5:]

In [1025]:
# indices_start = [indices_start[0], indices_start[-2], indices_start[-1]]
# indices_end = [indices_end[0], indices_end[-2], indices_end[-1]]

In [1026]:
disrupt_errors = []
disrupt_predictions = []
disrupt_true_values = []

non_disrupt_errors = []
non_disrupt_predictions = []
non_disrupt_true_values = []

for s in tqdm(range(len(indices_start))):
    start = indices_start[s] + 1
    end = indices_end[s] + 1

    shot_number = df[start:end]['shot'].unique()
    assert len(shot_number) == 1, f"shots are not unique for s={s}"

    x = r_matrix[-1]

    states = []
#     data = testing_input[start:end, 1:]
    for i in data:
        x = _update_state(
            A=A,
            W_in=W_in,
            state=x,
            inputs=i
        )
        states.append(x)

    model_predictions = []
    for model in all_models:
        model_predictions.append(
            model.predict(states)
        )
    predictions = np.mean(model_predictions, axis=0)

    score = mean_squared_error(
                testing_output[start:end, 1:],
                predictions, 
                multioutput='raw_values')

    if comparison[comparison.shot == shot_number[0]]['variable'].values[0] == 'disrupt':
        disrupt_errors.append(
           score
        )
        disrupt_predictions.append(
            predictions
        )
        disrupt_true_values.append(
            testing_output[start:end, 1:]
        )


    if comparison[comparison.shot == shot_number[0]]['variable'].values[0] == 'no_disrupt':
        non_disrupt_errors.append(
           score
        )
        non_disrupt_predictions.append(
            predictions
        )
        non_disrupt_true_values.append(
            testing_output[start:end, 1:]
        )

100%|██████████| 20/20 [02:11<00:00,  6.57s/it]


In [1027]:
print('disrupt mse: ', np.mean(disrupt_errors))
print('non disrupt mse: ', np.mean(non_disrupt_errors))

disrupt mse:  1.0099294192612156
non disrupt mse:  1.2738493125202142


In [5]:
# for i in range(len(disrupt_predictions)):
#     y_pred_zero = pd.DataFrame(disrupt_predictions[i][:, 0], columns=['ip'])    
#     y_true_zero = pd.DataFrame(disrupt_true_values[i][:, 0], columns=['ip'])    
    
#     plt.plot(y_pred_zero, label='pred', color='red')
#     plt.plot(y_true_zero, label='true', color='blue')

#     plt.legend()
#     plt.title('ip')
#     plt.show();

In [6]:
# for i in range(len(non_disrupt_predictions)):
#     y_pred_zero = pd.DataFrame(non_disrupt_predictions[i][:, 0], columns=['ip'])    
#     y_true_zero = pd.DataFrame(non_disrupt_true_values[i][:, 0], columns=['ip'])    
    
#     plt.plot(y_pred_zero, label='pred', color='red')
#     plt.plot(y_true_zero, label='true', color='blue')

#     plt.legend()
#     plt.title('ip')
#     plt.show();